# static

In [ ]:
# TRAIN_PATH = '/content/drive/MyDrive/student_cup_2021/dataset/train.csv'
# TEST_PATH = '/content/drive/MyDrive/student_cup_2021/dataset/test.csv'

# for new dataframe
TRAIN_PATH = '/content/drive/MyDrive/student_cup_2021/dataset/new_train.csv'
TEST_PATH = '/content/drive/MyDrive/student_cup_2021/dataset/new_test.csv'

MODEL_PATH = 'best_model.h5'

# function

# Loading data

In [ ]:
import pandas as pd

# df_train = pd.read_csv(TRAIN_PATH)
# df_test = pd.read_csv(TEST_PATH)

# for new dataframe
df_train = pd.read_csv(TRAIN_PATH, index_col=0)
df_test = pd.read_csv(TEST_PATH, index_col=0)

df = pd.concat([df_train.drop('genre', axis=1), df_test], axis=0)

print('df shape: {0}'.format(df.shape))

df shape: (8092, 13)


# Preprocessing

## 欠損値

In [ ]:
# リストワイズ法
# df_train = df_train.dropna()
# df_train = df_train.reset_index(drop=True)

# print('df_train shape: {0}'.format(df_train.shape))

## tempo

In [ ]:
def tempo_preprocessing(data):
  
  df_temp = data.str.split('-', expand=True).astype(int)
  df_temp.columns = ['tempo_min', 'tempo_max']

  return df_temp

df = pd.concat([df, tempo_preprocessing(df['tempo'])], axis=1)

print('df shape: {0}'.format(df.shape))

df shape: (8092, 15)


## duration_ms

In [ ]:
# import numpy as np

# df['duration_s'] = df['duration_ms'] / 1000
# df['duration_m'] = df['duration_ms'] / 60000
# df['duration_log10'] = np.log10(df['duration_ms']+1)

# print('df_train shape: {0}'.format(df_train.shape))

## <font color='red'>One-hot-encoding</font>

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

df['popularity_temp'] = df['popularity']

# popularity, tempo, region
ohe_list = ['popularity_temp', 'tempo', 'region']
for col in ohe_list:
  le = LabelEncoder()
  le.fit(df[col]) # df_train[col] -> df[col]
  df[col] = le.transform(df[col])
  df[col] = col+'_'+df[col].astype(str)
  df = pd.concat([df, pd.get_dummies(df[col])], axis=1)
  
print('df shape: {0}'.format(df.shape))

df shape: (8092, 132)


## Target

In [ ]:
df.head()

,index,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region,tempo_min,tempo_max,popularity_temp,popularity_temp_0,popularity_temp_1,popularity_temp_10,popularity_temp_11,popularity_temp_12,popularity_temp_13,popularity_temp_14,popularity_temp_15,popularity_temp_16,popularity_temp_17,popularity_temp_18,popularity_temp_19,popularity_temp_2,popularity_temp_20,popularity_temp_21,popularity_temp_22,popularity_temp_23,popularity_temp_24,popularity_temp_25,popularity_temp_26,popularity_temp_27,popularity_temp_28,popularity_temp_29,popularity_temp_3,...,popularity_temp_78,popularity_temp_79,popularity_temp_8,popularity_temp_80,popularity_temp_81,popularity_temp_82,popularity_temp_9,tempo_0,tempo_1,tempo_10,tempo_11,tempo_2,tempo_3,tempo_4,tempo_5,tempo_6,tempo_7,tempo_8,tempo_9,region_0,region_1,region_10,region_11,region_12,region_13,region_14,region_15,region_16,region_17,region_18,region_19,region_2,region_20,region_3,region_4,region_5,region_6,region_7,region_8,region_9
0,0,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,tempo_1,region_7,121,152,popularity_temp_11,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,tempo_2,region_8,153,176,popularity_temp_69,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,2,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,tempo_9,region_4,64,76,popularity_temp_43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,3,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,tempo_3,region_2,177,192,popularity_temp_45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,4,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,tempo_11,region_20,97,120,popularity_temp_57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
df_genre = df_train['genre']

print('df shape: {0}, df_genre shape: {1}'.format(df.shape, df_genre.shape))

df shape: (8092, 132), df_genre shape: (4046,)


## ペアワイズ

In [ ]:
# df_train = df_train.drop(['index', 'genre', 'popularity_temp', 'tempo', 'region'], axis=1)
df = df.drop(['index', 'popularity_temp', 'tempo', 'region'], axis=1)
# df = df.drop(['index', 'genre', 'tempo', 'region'], axis=1)

print('df shape: {0}'.format(df_train.shape))

df shape: (4046, 14)


## relationship

In [ ]:
# df['tempo_min_bpm'] = df['tempo_min']*(df['duration_ms'] / 60000)
# df['tempo_max_bpm'] = df['tempo_max']*(df['duration_ms'] / 60000)
# df['tempo_avg_bpm'] = ((df['tempo_min']+df['tempo_max']) / 2)*(df['duration_ms'] / 60000)

# print('df_train shape: {0}'.format(df_train.shape))

In [ ]:
# from itertools import combinations

# col_list = ['acousticness', 'positiveness', 'danceability', 'energy', 'liveness', 'speechiness', 'instrumentalness']

# for a, b in combinations(col_list, 2):
#   df[a+'_plus_'+b] = (df[a]+df[b]) / 2
#   df[a+'_times_'+b] = df[a]*df[b]*2

# for a, b, c in combinations(col_list, 3):
#   df[a+'_plus_'+b+'_plus_'+c] = (df[a]+df[b]+df[c]) / 3
#   df[a+'_times_'+b+'_times_'+c] = df[a]*df[b]*df[c]*3

# for a, b, c, d in combinations(col_list, 4):
#   df[a+'_plus_'+b+'_plus_'+c+'_plus_'+d] = (df[a]+df[b]+df[c]+df[d]) / 4
#   df[a+'_times_'+b+'_times_'+c+'_times_'+d] = df[a]*df[b]*df[c]*df[d]*4

# for a, b, c, d, e in combinations(col_list, 5):
#   df[a+'_plus_'+b+'_plus_'+c+'_plus_'+d+'_plus_'+e] = (df[a]+df[b]+df[c]+df[d]+df[e]) / 5
#   df[a+'_times_'+b+'_times_'+c+'_times_'+d+'_times_'+e] = df[a]*df[b]*df[c]*df[d]*df[e]*5

# for a, b, c, d, e, f in combinations(col_list, 6):
#   df[a+'_plus_'+b+'_plus_'+c+'_plus_'+d+'_plus_'+e+'_plus_'+f] = (df[a]+df[b]+df[c]+df[d]+df[e]+df[f]) / 6
#   df[a+'_times_'+b+'_times_'+c+'_times_'+d+'_times_'+e+'_times_'+f] = df[a]*df[b]*df[c]*df[d]*df[e]*df[f]*6

# for a, b, c, d, e, f, g in combinations(col_list, 7):
#   df[a+'_plus_'+b+'_plus_'+c+'_plus_'+d+'_plus_'+e+'_plus_'+f+'_plus_'+g] = (df[a]+df[b]+df[c]+df[d]+df[e]+df[f]+df[g]) / 7
#   df[a+'_times_'+b+'_times_'+c+'_times_'+d+'_times_'+e+'_times_'+f+'_times_'+g] = df[a]*df[b]*df[c]*df[d]*df[e]*df[f]*df[g]*7

# print('df shape: {0}'.format(df.shape))

## <font color='red'>正規化</font>

### <font color='red'>Min-Max</font>

In [ ]:
# # for pandas
# col_list = ['popularity', 'duration_ms', 'loudness', 'tempo_min', 'tempo_max']

# for col in col_list:
#   df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

# print('df_train shape: {0}'.format(df.shape))

### <font color='red'>Standardization</font>

In [ ]:
# for pandas
col_list = ['popularity', 'duration_ms', 'loudness', 'tempo_min', 'tempo_max']
# col_list = ['popularity', 'duration_ms', 'loudness', 'tempo_min', 'tempo_max', 'tempo_avg_bpm']

for col in col_list:
  df[col] = (df[col] - df[col].mean()) / df[col].std()

print('df_train shape: {0}'.format(df.shape))

df_train shape: (8092, 128)


In [ ]:
# df_train.head()

### <font color='red'>robust standardization</font>

## 正規化確認

In [ ]:
df.describe()

,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo_min,tempo_max,popularity_temp_0,popularity_temp_1,popularity_temp_10,popularity_temp_11,popularity_temp_12,popularity_temp_13,popularity_temp_14,popularity_temp_15,popularity_temp_16,popularity_temp_17,popularity_temp_18,popularity_temp_19,popularity_temp_2,popularity_temp_20,popularity_temp_21,popularity_temp_22,popularity_temp_23,popularity_temp_24,popularity_temp_25,popularity_temp_26,popularity_temp_27,popularity_temp_28,popularity_temp_29,popularity_temp_3,popularity_temp_30,popularity_temp_31,popularity_temp_32,popularity_temp_33,...,popularity_temp_78,popularity_temp_79,popularity_temp_8,popularity_temp_80,popularity_temp_81,popularity_temp_82,popularity_temp_9,tempo_0,tempo_1,tempo_10,tempo_11,tempo_2,tempo_3,tempo_4,tempo_5,tempo_6,tempo_7,tempo_8,tempo_9,region_0,region_1,region_10,region_11,region_12,region_13,region_14,region_15,region_16,region_17,region_18,region_19,region_2,region_20,region_3,region_4,region_5,region_6,region_7,region_8,region_9
count,8.092000e+03,8.092000e+03,8092.000000,8092.000000,8092.000000,8.092000e+03,8092.000000,8092.000000,8092.000000,8092.000000,8.092000e+03,8.092000e+03,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.00000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,...,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000,8092.000000
mean,1.984050e-16,5.781097e-17,0.340824,0.466998,0.501080,2.423735e-15,0.606151,0.267525,0.200370,0.215541,-6.977996e-17,3.149291e-16,0.004449,0.003089,0.004572,0.004572,0.005067,0.004943,0.005067,0.00655,0.007538,0.010257,0.009021,0.009516,0.002966,0.011246,0.010381,0.010010,0.013594,0.010751,0.011987,0.011987,0.014212,0.014335,0.015942,0.003337,0.018413,0.016930,0.019155,0.016930,...,0.004449,0.005314,0.003955,0.002966,0.001236,0.000247,0.006179,0.002472,0.345156,0.183144,0.282625,0.107143,0.031760,0.012481,0.000618,0.000247,0.000989,0.003584,0.029783,0.002224,0.083169,0.057093,0.027805,0.000371,0.009268,0.035838,0.082180,0.009145,0.010504,0.015818,0.042635,0.021132,0.086011,0.050544,0.167202,0.036456,0.013470,0.047578,0.194390,0.007168
std,1.000000e+00,1.000000e+00,0.238695,0.223599,0.160348,1.000000e+00,0.200901,0.156326,0.085047,0.155850,1.000000e+00,1.000000e+00,0.066555,0.055501,0.067469,0.067469,0.071005,0.070138,0.071005,0.08067,0.086501,0.100763,0.094557,0.097089,0.054383,0.105454,0.101361,0.099554,0.115804,0.103136,0.108834,0.108834,0.118369,0.118876,0.125258,0.057671,0.134449,0.129018,0.137077,0.129018,...,0.066555,0.072707,0.062764,0.054383,0.035134,0.015720,0.078368,0.049657,0.475448,0.386808,0.450303,0.309314,0.175371,0.111028,0.024851,0.015720,0.031429,0.059761,0.169997,0.047114,0.276154,0.232035,0.164425,0.019252,0.095831,0.185897,0.274656,0.095196,0.101957,0.124779,0.202045,0.143833,0.280397,0.219078,0.373179,0.187433,0.115284,0.212884,0.395754,0.084363
min,-2.547607e+00,-2.886521e+00,0.000000,0.000000,0.000000,-7.443157e+00,0.000000,0.000000,0.000000,0.000000,-3.819665e+00,-3.062315e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00

## train data & test data

In [ ]:
from keras.utils import to_categorical

X_train = df.iloc[:4046, :].values
X_test = df.iloc[4046:, :].values

Y_train = df_genre.values # Triplet+knn, LightGBM
cY_train = to_categorical(df_genre)

print('X_train shape: {0}, cY_train shape: {1}, Y_train shape: {2}'.format(X_train.shape, cY_train.shape, Y_train.shape))

X_train shape: (4046, 128), cY_train shape: (4046, 11), Y_train shape: (4046,)


# Creating neural network model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, LeakyReLU
from keras.optimizers import Adam

def create_model():

  model = Sequential()

  model.add(Dense(200, input_shape=(X_train.shape[1],)))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))

  model.add(Dense(100))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))
  
  model.add(Dense(11, activation='softmax'))
 
  model.compile(optimizer=Adam(lr=1e-5),
                # loss='categorical_crossentropy',
                loss = tfa.losses.SigmoidFocalCrossEntropy(),
                metrics=['accuracy'],) # tfa.metrics.F1Score(average='macro', num_classes=11)
  
  return model

# Creating Triplet model

In [ ]:
!pip install tensorflow_addons

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, LeakyReLU, Lambda
from keras.optimizers import Adam
import tensorflow as tf
import tensorflow_addons as tfa

def create_triplet_model():

  model = Sequential()

  model.add(Dense(200, input_shape=(X_train.shape[1],)))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=1e-2))
  
  model.add(Dense(100, activation=None))

  model.add(Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))

  model.compile(optimizer=Adam(),
                loss=tfa.losses.TripletSemiHardLoss(),)
  
  return model

  Using cached https://files.pythonhosted.org/packages/74/e3/56d2fe76f0bb7c88ed9b2a6a557e25e83e252aec08f13de34369cd850a0b/tensorflow_addons-0.12.1-cp37-cp37m-manylinux2010_x86_64.whl


# k-Fold cross validation

- 全ての機械学習モデルにおいて,入力させる列の順番は予測に大きく影響を与えることから,One-hot-encodingにおいては気を付ける必要がある.

|dataframe|scaler|model|cv score|notes|
|:--:|:--:|:--:|:--:|:--:|
|train|min-max scaler|nn|0.567||
|new train|min-max scaler|nn|0.548||
|train|standardization scaler|nn|0.552|精度はmin-maxよりも劣っているが,cvでの各予測精度は安定している|
|train|standardization scaler|LightGBM|0.538|parameter tuningで精度は上がりそうだが限界がありそう|
|train|standardization scaler|triplet+knn|ゴミ||
|train|standardization scaler|knn|0.479 (n_neightbors=1)||
|train|standardization scaler|nn|0.560|one-hot-encodingバグ修正|
|train|min-max scaler|nn|0.551|one-hot-encodingバグ修正 & StratifiedKFold|
|train|standardization scaler|LightGBM|0.531|one-hot-encodingバグ修正|
|train|standardization scaler|knn|0.479|one-hot-encodingバグ修正|
|train|standardization scaler (a part of)|nn|0.570||
|train|standardization scaler (a part of)|nn|0.566|ModelCheckpoint|
|train|standardization scaler (a part of)|nn|0.563|epochs -> 200|
|train|standardization scaler (a part of)|nn|0.553|SigmoidFocalCrossEntropy, epochs=100|
|train|standardization scaler (a part of)|Triplet+knn|0.487(n_neighbors=1)||
|train|standardization scaler (a part of)|nn|0.547|no to_categorical(popularity)|
|train|standardization scaler (a part of)|nn|0.579-0.576|focal loss, epochs=200|

## neural network

In [ ]:
# from keras.callbacks import EarlyStopping

# model = create_model()
# history = model.fit(X_train, cY_train, 
#                     epochs=200, 
#                     batch_size=4,
#                     verbose=1,
#                     validation_split=0.2)

In [ ]:
# import matplotlib.pyplot as plt

# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(1, len(acc) + 1)

# plt.plot(epochs, acc, 'b', label='Training acc')
# plt.plot(epochs, val_acc, 'b', color='orange', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()

# plt.show()

# plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'b', color='orange', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

# plt.show()

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import f1_score
import numpy as np

class CustomCallback(Callback):

  def __init__(self, model, x_val, cy_val, model_path):
    self.model = model
    self.x_val = x_val
    self.cy_val = cy_val
    self.model_path = model_path
    self.max = 0
  
  def on_epoch_end(self, epoch, logs=None):
    score = f1_score(np.argmax(self.cy_val, axis=1), np.argmax(self.model.predict(self.x_val), axis=1), average='macro')
    if self.max < score:
      self.max = score
      self.model.save(self.model_path)

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.metrics import f1_score

def cross_val_score_for_keras(X_train, Y_train, epochs, batch_size, n_splits=10):

  skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2021)

  loss=[]
  acc=[]
  f1_macro = []
  model_list = []

  num = 0

  for train_idx, val_idx in skf.split(X_train, cY_train.argmax(axis=1)):
    train_data = X_train[train_idx]
    train_labels = cY_train[train_idx]
    val_data = X_train[val_idx]
    val_labels = cY_train[val_idx]

    model = create_model()
    model_path = str(num)+'_'+MODEL_PATH
    callbacks_list = [CustomCallback(model, val_data, val_labels, model_path),]
    history = model.fit(train_data, train_labels, 
                        epochs=epochs, 
                        batch_size=batch_size, 
                        callbacks=callbacks_list, 
                        verbose=0, 
                        validation_data=(val_data, val_labels))
    model_list.append(load_model(model_path))

    score = f1_score(np.argmax(val_labels, axis=1), np.argmax(model_list[num].predict(val_data), axis=1), average='macro')
    f1_macro.append(score)
    print('f1 macro: {0:.3f}'.format(score))

    num += 1

  return f1_macro, model_list

from time import time

EPOCHS = 500
BATCH_SIZE = 4

start_time = time()
f1_macro, model_list = cross_val_score_for_keras(X_train, cY_train, EPOCHS, BATCH_SIZE, 10)
elapsed_time = time() - start_time

print('Elapsed time: {0:.3f} m'.format(elapsed_time / 60))
print('f1 macro for cv: {0:.3f}'.format(np.mean(f1_macro)))

f1 macro: 0.608
f1 macro: 0.559
f1 macro: 0.493
f1 macro: 0.610
f1 macro: 0.516
f1 macro: 0.612
f1 macro: 0.584
f1 macro: 0.514
f1 macro: 0.518
f1 macro: 0.556
Elapsed time: 135.305 m
f1 macro for cv: 0.557


In [ ]:
# skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=2021)

# f1_macro = []

# num = 1

# for train_idx, val_idx in skf.split(X_train, cY_train.argmax(axis=1)):
#   train_data = X_train[train_idx]
#   train_labels = cY_train[train_idx]
#   val_data = X_train[val_idx]
#   val_labels = cY_train[val_idx]

#   # model = create_model()
#   # model_path = str(num)+'_'+MODEL_PATH
#   # callbacks_list = [ModelCheckpoint(filepath=model_path, monitor='val_loss', save_best_only=True, mode='min',)]
#   # model = load_model(model_path)

#   score = f1_score(np.argmax(val_labels, axis=1), 
#                    np.argmax(
#                        0.1*model_list[0].predict(val_data)
#                        +0.1*model_list[2].predict(val_data)
#                        +0.1*model_list[3].predict(val_data)
#                        +0.1*model_list[4].predict(val_data)
#                        +0.1*model_list[5].predict(val_data)
#                        +0.1*model_list[6].predict(val_data)
#                        +0.1*model_list[7].predict(val_data)
#                        +0.1*model_list[8].predict(val_data)
#                        +0.1*model_list[9].predict(val_data), axis=1), average='macro')
#   f1_macro.append(score)
#   print('f1 macro: {0}'.format(score))

#   num += 1

# print('f1 macro (cv=10): {0:.3f}'.format(np.mean(f1_macro)))

## Triplet model + knn

In [ ]:
# from time import time
# from keras.callbacks import ModelCheckpoint

# EPOCHS = 500
# BATCH_SIZE = 256

# start = time()

# callbacks_list = [ModelCheckpoint(filepath=MODEL_PATH, monitor='loss', save_best_only=True, mode='min',)]

# model_T = create_triplet_model()
# history = model_T.fit(X_train, Y_train,
#                       epochs=EPOCHS,
#                       batch_size=BATCH_SIZE,
#                       callbacks=callbacks_list,
#                       verbose=0,)

# elapsed_time = time() - start
# print('Elapsed time: {0:.3f} ms'.format(elapsed_time / 60))

In [ ]:
# import matplotlib.pyplot as plt

# loss = history.history['loss']

# epochs = range(1, len(loss) + 1)

# plt.plot(epochs, loss, 'b', label='Training loss')

# plt.title('Training loss')
# plt.legend()

# plt.show()

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import cross_val_score
# from keras.models import load_model

# model_T = load_model(MODEL_PATH)

# for num in range(1, 11):

#   clf = KNeighborsClassifier(n_neighbors=num)

#   # scores = cross_val_score(clf, model_T.predict(X_train), Y_train, scoring='f1_macro', cv=10)
#   scores = cross_val_score(clf, X_train, Y_train, scoring='f1_macro', cv=10)

#   print('n_neighbors='+str(num)+', f1 macro of cross validation: {0:.3f}'.format(scores.mean()))

## LightGBM

In [ ]:
# import lightgbm as lgb
# from sklearn.model_selection import cross_val_score

# clf = lgb.LGBMClassifier()

# scores = cross_val_score(clf, X_train, Y_train, scoring='f1_macro', cv=10)

# print('f1 macro (CV=10): {0:.3f}'.format(scores.mean()))

## knn

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import cross_val_score

# for num in range(1, 11):

#   clf = KNeighborsClassifier(n_neighbors=num)

#   scores = cross_val_score(clf, X_train, Y_train, scoring='f1_macro', cv=10)

#   print('n_neighbors='+str(num)+', f1 macro of cross validation: {0:.3f}'.format(scores.mean()))

# training model

# sumission

In [ ]:
predicted =  np.argmax(np.mean([model_list[0].predict(X_test),
                               model_list[2].predict(X_test),
                               model_list[3].predict(X_test),
                               model_list[4].predict(X_test),
                               model_list[5].predict(X_test),
                               model_list[6].predict(X_test),
                               model_list[7].predict(X_test),
                               model_list[8].predict(X_test),
                               model_list[9].predict(X_test)], axis=0), axis=1)

# from scipy import stats
# predicted =  np.argmax(stats.hmean(model_list[0].predict(X_test)
#                                    +model_list[2].predict(X_test)
#                                    +model_list[3].predict(X_test)
#                                    +model_list[4].predict(X_test)
#                                    +model_list[5].predict(X_test)
#                                    +model_list[6].predict(X_test)
#                                    +model_list[7].predict(X_test)
#                                    +model_list[8].predict(X_test)
#                                    +model_list[9].predict(X_test), axis=1), axis=1)

df_sub = pd.concat([df_test['index'].reset_index(drop=True), pd.DataFrame(predicted, columns=['predicted']).astype(int)], axis=1)
df_sub.to_csv('am_submission.csv', index=False, header=False)

df_sub.head()

,index,predicted
0,4046,7
1,4047,10
2,4048,10
3,4049,10
4,4050,8


In [ ]:
from scipy import stats

predicted =  np.argmax(stats.hmean([model_list[0].predict(X_test),
                               model_list[2].predict(X_test),
                               model_list[3].predict(X_test),
                               model_list[4].predict(X_test),
                               model_list[5].predict(X_test),
                               model_list[6].predict(X_test),
                               model_list[7].predict(X_test),
                               model_list[8].predict(X_test),
                               model_list[9].predict(X_test)], axis=0), axis=1)

df_sub = pd.concat([df_test['index'].reset_index(drop=True), pd.DataFrame(predicted, columns=['predicted']).astype(int)], axis=1)
df_sub.to_csv('hm_submission.csv', index=False, header=False)

df_sub.head()

,index,predicted
0,4046,7
1,4047,10
2,4048,10
3,4049,10
4,4050,8


# Colab

```javascript
function ClickConnect(){ 
console.log("Working"); 
document.querySelector("#comments > span").click()
}
setInterval(ClickConnect,500000)
```